In [ ]:
# Para que las funciones se actualicen si tener que refrescar el kernel
%load_ext autoreload
%autoreload 2

In [ ]:
# Funciones propias
from utils_limpieza import * 

In [ ]:
import pandas as pd
import json
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt

In [ ]:
# Cargar datos
path = '../data/raw/tp2_muestra_diplodatos_ventas_omega_modelo_2023.csv'
ventas = pd.read_csv(path)


## Hacemos la limpieza sugerida al principio del TP2
- [x] Eliminar las variables en `cols_dropear` (Paso 1).
- [x] Renombrar a 'Otros' las subcategorías que no tengan `MODELO` (Paso 2).

In [ ]:
print(f'El df original tiene {ventas.shape[0]} registros y {ventas.shape[1]} variables.\n')

# Paso 1
cols_dropear = ['INSCRIPCION', 'CATEGORIA', 'DESCRIPCION_CATEGORIA', 'CATEGORIA (Ajustado)', 'NOMBRE']
ventas_renamed = limpiar_basic(ventas, 
                             cols_drop=cols_dropear)
print(f'Luego del paso 1 quedan se tiran {len(cols_dropear)} variables, quedando {ventas_renamed.shape[1]} variables.\n')

# Paso 2
# ventas_renamed['SUB-CATEGORIA'].nunique() # antes del paso 2
ventas_renamed = renombrar_elementos(ventas_renamed, 
                                     columna='SUB-CATEGORIA', 
                                     fill_otros='Otros')

scu_vr = ventas_renamed['SUB-CATEGORIA'].nunique()
scu_ot = ventas_renamed['SUB-CATEGORIA'].nunique() - scu_vr + 1
print(f'Luego del paso 2 quedan {scu_vr} subcategorías únicas.')
print(f'Las {scu_ot} subcategorías únicas que no tenían ningún vendedor modelo fueron agrupadas en "Otros".')
print(f'Las {scu_vr-1} subcategorías que sí tienen vendedores modelo son:')

ventas_renamed.groupby(by=['SUB-CATEGORIA'])['MODELO'].sum().sort_values(ascending=False)[:12]

## Anonimizado + simplificación de variables y valores
- [x] Reordenamos arbitrariamente las columnas.
- [x] Simplificamos el nombre de las variables.
- [x] Anonimizamos la variable sensible `ID` (Paso 3).
- [x] Simplificamos valores en `Deposito`.
- [x] Guardamos todos estos diccionarios.

In [ ]:
# Se reacomodan las columnas
ventas_renamed = ventas_renamed[['ID_VENDEDOR', 'OMEGA', 'SUB-CATEGORIA', 
                                 'DESC_TRATAMIENTO_FISCAL', 'TRATAMIENTO_FISCAL',
                                 'TRATAMIENTO_DIFERNCIAL', 'CM04', 'AÑO', 'MES', 
                                 'DEPOSITO', 'TOTAL_VENTAS', 
                                 'PORCENTAJE_COMISION_EMPRESA', 
                                 'COMISION_EMPRESA', 'MODELO']]

ventas_renamed = ventas_renamed.sort_values(['AÑO', 'MES', 'TOTAL_VENTAS']).reset_index(drop=True)

In [ ]:
# Se modifican los nombres de las columnas

# Creamos el diccionario
column_dict = {'ID_VENDEDOR': 'ID', 
                'OMEGA': 'Omega', 
                'SUB-CATEGORIA': 'Subrubro', 
                'DESC_TRATAMIENTO_FISCAL': 'Trat_Fisc_Agg', 
                'TRATAMIENTO_FISCAL': 'Trat_Fisc', 
                'TRATAMIENTO_DIFERNCIAL': 'Trat_Dif', 
                'CM04': 'CM', 
                'AÑO': 'Año', 
                'MES': 'Mes', 
                'DEPOSITO': 'Deposito', 
                'TOTAL_VENTAS': 'Ventas', 
                'PORCENTAJE_COMISION_EMPRESA': 'Alicuota', 
                'COMISION_EMPRESA': 'Recaudacion', 
                'MODELO': 'Modelo'}

with open("../references/tp2_column_dict.json", "w") as fp:
    json.dump(column_dict , fp)

# Renombramos
with open("../references/tp1_column_dict.json") as column_dict_json:
    column_dict = json.load(column_dict_json)

ventas_renamed.rename(columns = column_dict, inplace = True)
ventas_renamed.columns

In [ ]:
# Anonimizamos y guardamos el diccionario para no perder esta info
ventas_hash, dict_id = anonimizar(ventas_renamed, 'ID')

with open("../references/tp2_ID_dict.json", "w") as fp:
    json.dump(dict_id, fp)

print('¡Diccionario generado!')

In [ ]:
# Mapeo de `Deposito` a enteros
ventas_hash, dict_dep = anonimizar(ventas_renamed, 'Deposito')

with open("../references/tp2_deposito_dict.json", "w") as fp:
    json.dump(dict_dep, fp)

print('¡Diccionario generado!')